In [1]:
!pip install pathway bokeh --quiet # This cell may take a few seconds to execute.

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import datetime
from datetime import datetime
import pathway as pw
import bokeh.plotting
import panel as pn

In [3]:
df = pd.read_csv('/content/data.csv')
df

,Unnamed: 0,SystemCodeNumber,Capacity,Occupancy,LastUpdatedDate,LastUpdatedTime,IsSpecialDay,VehicleType,Latitude,Longitude,TrafficConditionNearby,QueueLength
0,0,BHMBCCMKT01,577,61,04-10-2016,07:59:42,0,car,28.5,77.15,low,2
1,1,BHMBCCMKT01,577,64,04-10-2016,08:25:42,0,car,28.5,77.15,average,2
2,2,BHMBCCMKT01,577,80,04-10-2016,08:59:42,0,car,28.5,77.15,low,2
3,3,BHMBCCMKT01,577,107,04-10-2016,09:32:46,0,car,28.5,77.15,low,3
4,4,BHMBCCMKT01,577,150,04-10-2016,09:59:48,0,car,28.5,77.15,low,3
...,...,...,...,...,...,...,...,...,...,...,...,...
1307,1307,BHMBCCMKT01,577,309,19-12-2016,14:30:33,0,bike,28.5,77.15,average,5
1308,1308,BHMBCCMKT01,577,300,19-12-2016,15:03:34,0,car,28.5,77.15,low,4
1309,1309,BHMBCCMKT01,577,274,19-12-2016,15:29:33,0,truck,28.5,77.15,low,3
1310,1310,BHMBCCMKT01,577,230,19-12-2016,16:03:35,0,cycle,28.5,77.15,low,2


In [26]:
df['Timestamp'] = pd.to_datetime(df['LastUpdatedDate'] + ' ' + df['LastUpdatedTime'],
                                  format='%d-%m-%Y %H:%M:%S')

df = df.sort_values('Timestamp').reset_index(drop=True)
df.to_csv("parking_stream_2.csv", index=False)

In [5]:
df[["Timestamp", "Occupancy", "Capacity"]].to_csv("parking_stream.csv", index=False)

In [7]:
class ParkingSchema(pw.Schema):
    Timestamp: str
    Occupancy: int
    Capacity: int

In [8]:
data = pw.demo.replay_csv("parking_stream.csv", schema=ParkingSchema, input_rate=1000)

In [9]:
fmt = "%Y-%m-%d %H:%M:%S"

data_with_time = data.with_columns(
    t = data.Timestamp.dt.strptime(fmt),
    day = data.Timestamp.dt.strptime(fmt).dt.strftime("%Y-%m-%dT00:00:00")
)

In [23]:
import datetime
import math

delta_window = (
    data_with_time.windowby(
        pw.this.t,
        instance=pw.this.day,
        window=pw.temporal.tumbling(datetime.timedelta(days=1)),
        behavior=pw.temporal.exactly_once_behavior()
    )
    .reduce(
        t=pw.this._pw_window_end,
        cap=pw.reducers.max(pw.this.Capacity),
        occ_avg = pw.reducers.sum(pw.this.Occupancy)/pw.reducers.count(),
    )
    .with_columns(
        price = 8 + 12 / (1 + pw.this.cap / ((pw.this.occ_avg)/2))
    )
)


In [24]:
pn.extension()

def price_plotter(source):
    fig = bokeh.plotting.figure(
        height=400,
        width=800,
        title="Pathway: Daily Parking Price",
        x_axis_type="datetime",
    )
    fig.line("t", "price", source=source, line_width=2, color="navy")
    fig.circle("t", "price", source=source, size=6, color="red")

    return fig

viz = delta_window.plot(price_plotter, sorting_col="t")
pn.Column(viz).servable()

Column
    [0] Column
        [0] Row
            [0] Markdown(str)
            [1] TooltipIcon(value='Table depends o...)
        [1] Bokeh(figure)

In [25]:
%%capture --no-display
pw.run()

Output()

In [64]:
import pathway as pw

class ParkingSchema(pw.Schema):
    Timestamp: str
    Occupancy: int
    Capacity: int
    QueueLength: int
    TrafficConditionNearby: str
    IsSpecialDay: int
    VehicleType: str


In [65]:
data = pw.io.csv.read("parking_stream_2.csv", schema=ParkingSchema, mode="static")

In [88]:
import datetime
from pathway.internals import dtype

VEHICLE_WEIGHTS = {
    "bike": 0.5,
    "car": 1.0,
    "truck": 1.5,
    "cycle": 0.2,
}

TRAFFIC_WEIGHTS = {
    "low": 1.0,
    "average": 2.5,
    "high": 5.0,
}

fmt = "%Y-%m-%d %H:%M:%S"

data_with_time = data.with_columns(
    t = data.Timestamp.dt.strptime(fmt),
    day = data.Timestamp.dt.strptime(fmt).dt.strftime("%Y-%m-%dT00:00:00"),
    vehicle_weight = int(VEHICLE_WEIGHTS.get(data.VehicleType, 1.0)),
    traffic_numeric = int(TRAFFIC_WEIGHTS.get(data.TrafficConditionNearby, 3.0))
)

data_with_time = data_with_time.with_columns(
    Occupancy = pw.cast(dtype.INT, data_with_time.Occupancy),
    Capacity = pw.cast(dtype.INT, data_with_time.Capacity),
    QueueLength = pw.cast(dtype.INT, data_with_time.QueueLength),
    IsSpecialDay = pw.cast(dtype.INT, data_with_time.IsSpecialDay),
    vehicle_weight = pw.cast(dtype.INT, data_with_time.vehicle_weight),
    traffic_numeric = pw.cast(dtype.INT, data_with_time.traffic_numeric),
)


In [97]:
MAX_QUEUE = 15
MAX_TRAFFIC = 5
LAMBDA = 0.5

delta_window = (
    data_with_time.windowby(
        pw.this.t,
        instance=pw.this.day,
        window=pw.temporal.tumbling(datetime.timedelta(days=1)),
        behavior=pw.temporal.exactly_once_behavior(),
    )
    .reduce(
        t = pw.this._pw_window_end,
        cap = pw.reducers.max(pw.this.Capacity),
        occ_sum = pw.reducers.sum(pw.this.Occupancy),
        queue_sum = pw.reducers.sum(pw.this.QueueLength),
        traffic_sum = pw.reducers.sum(pw.this.traffic_numeric),
        special = pw.reducers.max(pw.this.IsSpecialDay),
        vehicle_weight_sum = pw.reducers.sum(pw.this.vehicle_weight),
        count = pw.reducers.count(),
    )
    .with_columns(
        occ_avg = pw.this.occ_sum / pw.this.count,
        queue_avg = pw.this.queue_sum / pw.this.count,
        traffic_avg = pw.this.traffic_sum / pw.this.count,
        vehicle_weight_avg = pw.this.vehicle_weight_sum / pw.this.count,
    )
    .with_columns(
        norm_occ = pw.this.occ_avg / pw.this.cap,
        norm_queue = pw.this.queue_avg / MAX_QUEUE,
        norm_traffic = pw.this.traffic_avg / MAX_TRAFFIC,
    )
    .with_columns(
        norm_demand = (
            1.0 * pw.this.norm_occ +
            0.05 * pw.this.norm_queue -
            0.1 * pw.this.norm_traffic +
            0.2 * pw.this.special +
            1.0 * pw.this.vehicle_weight_avg
        ) / 6.0
    )
    .with_columns(
        price = pw.apply(lambda p: max(min(p, 20.0), 5.0), 10 * (1 + LAMBDA * pw.this.norm_demand))
    )
)

In [98]:
pn.extension()
viz = delta_window.plot(price_plotter, sorting_col="t")
pn.Column(viz).servable()

Column
    [0] Column
        [0] Row
            [0] Markdown(str)
            [1] TooltipIcon(value='Immediate table p...)
        [1] Bokeh(figure)

In [99]:
pw.run()

Output()